In [1]:
import time
import numpy as np
import pandas as pd
import statsmodels.api as sm
import utils
import missForest
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_org = pd.read_csv("../Example/data/original.csv")
df = pd.read_csv("../Example/data/missing.csv")
display(df)

C:\Users\kogby\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\kogby\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,PM10,PM2.5,RH,SO2
0,16.0,2.0,0.53,0.18,-0.30,16.0,16.0,106.0,105.4,77.0,2.6
1,16.0,2.0,0.56,0.19,0.00,23.0,23.0,106.0,97.6,77.0,3.8
2,16.0,2.0,0.56,0.20,0.10,23.0,23.0,107.0,97.6,79.0,3.6
3,16.0,2.1,0.61,0.22,1.72,19.2,20.9,108.0,98.9,80.0,3.7
4,16.0,2.1,0.67,0.19,-0.40,19.0,18.0,105.0,110.6,80.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...
8779,20.0,1.9,0.30,0.09,1.30,12.0,13.0,63.0,23.5,86.0,1.7
8780,20.0,1.9,0.28,0.09,1.40,13.0,14.0,55.0,26.1,87.0,1.6
8781,20.0,2.0,0.33,0.11,1.30,14.0,16.0,50.0,28.7,89.0,1.5
8782,19.0,2.0,0.37,0.12,1.30,19.0,20.0,52.0,19.6,89.0,1.6


In [2]:
y= df['PM2.5'].values
df.drop("PM2.5",axis=1, inplace=True)
missing_result = utils.simulate_nan(df, nan_rate=0.3)
mis_df = missing_result['X']

In [3]:
mf_imputer = missForest.MissForest(max_iter = 20)
start_time = time.time()
df_imp = mf_imputer.fit_transform(mis_df, verbose=False)
print(f"Execution time: {time.time() - start_time}")

Execution time: 16.508328914642334


In [4]:
X = sm.add_constant(df_imp)
display(X)
model = sm.OLS(y,X).fit()
print(model.summary())

,const,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,PM10,RH,SO2
0,1.0,16.000000,2.000000,0.553206,0.180000,-0.300000,19.140675,19.048008,106.000000,77.000000,2.600000
1,1.0,16.000000,2.000000,0.560000,0.190000,0.000000,23.000000,23.000000,109.878194,73.843214,3.800000
2,1.0,18.770788,2.000000,0.560000,0.200000,0.100000,23.000000,23.000000,107.000000,79.000000,3.600000
3,1.0,16.000000,1.909183,0.566300,0.220000,1.720000,19.200000,21.260207,97.200291,69.944822,3.700000
4,1.0,16.000000,2.100000,0.670000,0.184249,0.115597,19.000000,18.000000,129.952960,80.000000,3.600000
...,...,...,...,...,...,...,...,...,...,...,...
8779,1.0,20.000000,1.900000,0.357654,0.090000,1.300000,12.000000,13.000000,63.000000,86.000000,1.700000
8780,1.0,20.000000,1.900000,0.356530,0.090000,0.881831,13.000000,14.000000,55.000000,83.962716,1.600000
8781,1.0,20.000000,2.000000,0.330000,0.110000,1.300000,14.000000,16.000000,50.000000,89.000000,1.882426
8782,1.0,19.000000,2.000000,0.415677,0.120000,1.008453,19.000000,20.000000,65.365472,90.707827,1.907860


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     1748.
Date:                Tue, 10 Oct 2023   Prob (F-statistic):               0.00
Time:                        11:03:20   Log-Likelihood:                -36487.
No. Observations:                8784   AIC:                         7.300e+04
Df Residuals:                    8773   BIC:                         7.307e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -27.8937      3.192     -8.738      0.0

In [5]:
df_org.drop('PM2.5',axis=1,inplace=True)
X = sm.add_constant(df_org)
display(X)
model = sm.OLS(y,X).fit()
print(model.summary())

,const,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,PM10,RH,SO2
0,1.0,16.0,2.0,0.53,0.18,-0.30,16.0,16.0,106.0,77.0,2.6
1,1.0,16.0,2.0,0.56,0.19,0.00,23.0,23.0,106.0,77.0,3.8
2,1.0,16.0,2.0,0.56,0.20,0.10,23.0,23.0,107.0,79.0,3.6
3,1.0,16.0,2.1,0.61,0.22,1.72,19.2,20.9,108.0,80.0,3.7
4,1.0,16.0,2.1,0.67,0.19,-0.40,19.0,18.0,105.0,80.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...
8779,1.0,20.0,1.9,0.30,0.09,1.30,12.0,13.0,63.0,86.0,1.7
8780,1.0,20.0,1.9,0.28,0.09,1.40,13.0,14.0,55.0,87.0,1.6
8781,1.0,20.0,2.0,0.33,0.11,1.30,14.0,16.0,50.0,89.0,1.5
8782,1.0,19.0,2.0,0.37,0.12,1.30,19.0,20.0,52.0,89.0,1.6


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     2843.
Date:                Tue, 10 Oct 2023   Prob (F-statistic):               0.00
Time:                        11:03:20   Log-Likelihood:                -34956.
No. Observations:                8784   AIC:                         6.993e+04
Df Residuals:                    8773   BIC:                         7.001e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -33.5717      2.471    -13.588      0.0

In [6]:
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(mis_df)
# IterativeImputer(random_state=0)
imp_iterative = imp_mean.transform(mis_df)

In [7]:
X = sm.add_constant(imp_iterative)
# display(X)
model = sm.OLS(y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     1748.
Date:                Tue, 10 Oct 2023   Prob (F-statistic):               0.00
Time:                        11:03:20   Log-Likelihood:                -36487.
No. Observations:                8784   AIC:                         7.300e+04
Df Residuals:                    8773   BIC:                         7.307e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -27.8937      3.192     -8.738      0.0